# Feature transformation and data cleaning

In [1]:
# %%capture
# %pip install -r ../../requirements.txt --user

In [2]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np

from IPython.display import clear_output

cwd = Path.cwd()  # Current directory
sys.path.append(str(cwd.parents[0]))

from sklearn.preprocessing import LabelEncoder, PowerTransformer, QuantileTransformer
from definitions import LocationConfig
from helper_function import df_null_values,fill_null_values
lc = LocationConfig()

In [3]:
# Read consolidated table
df_raw_data = pd.read_csv(lc.feature_consolidation_output_table)
df_raw_data_null = df_raw_data.copy()

In [4]:
df_raw_data_null.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,Customer_Name,Province,Region,Customer_Segment
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,AARON BERGMAN,ALBERTA,WEST,CORPORATE
1,Ord_5406,Prod_13,SHP_7549,Cust_1818,42.27,0.01,13,4.56,0.93,0.54,AARON BERGMAN,ALBERTA,WEST,CORPORATE
2,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,AARON BERGMAN,ALBERTA,WEST,CORPORATE
3,Ord_5456,Prod_6,SHP_7625,Cust_1818,2337.89,0.09,43,729.34,14.30,0.37,AARON BERGMAN,ALBERTA,WEST,CORPORATE
4,Ord_5485,Prod_17,SHP_7664,Cust_1818,4233.15,0.08,35,1219.87,26.30,0.38,AARON BERGMAN,ALBERTA,WEST,CORPORATE


In [5]:
# Identiy non feature columns which needs to be removed 
columns_to_drop = [ ]
                  
# Drop the columns which won't contribute to feature selection
df_raw_data_null.drop(columns_to_drop, axis=1, inplace=True)

### Find null features and get the statistical measures for those null features

In [6]:
%%time
"""
Description:
    For ckecking null features and data point count
"""
for i in df_raw_data_null.columns:
    if df_raw_data_null[i].isnull().any():
        print(i.rjust(42), ' '.rjust(10),'data_type',df_raw_data_null[i].dtypes,' '.rjust(10), 'null->',df_raw_data_null[i].isnull().sum(), ' '.rjust(10),'non-null->',df_raw_data_null[i].notnull().sum())   

                       Product_Base_Margin            data_type float64            null-> 63            non-null-> 8336
CPU times: total: 0 ns
Wall time: 6 ms


In [7]:
# Creating dataframe for null values
statistics_dict = {'Mean':[ ], # Mention columns to fill with mean value
                   
                   'Median':[ ], # Mention columns to fill with median value
                   
                   'Mode':[ ], # Mention columns to fill with mode value
                  
                   'Zero':[ ]} # Mention columns to fill with 0 value


    
df_null_features = df_null_values(df_raw_data_null, statistics_dict)

In [9]:
# Save the null values data
df_null_features.to_csv(lc.null_features_agg_value,index=False)

In [10]:
# filling the null value

df_transformed = fill_null_values(df_raw_data, df_null_features)

### Constructing the Train/Test and OTT data

In [12]:
%%time
"""
Description:
    - Fitering the train/test and oot data based on the date for each window period
"""

df_transformed_train = df_raw_data[df_raw_data['Date_Column'] <= '']
df_transformed_oot = df_raw_data[df_raw_data['Date_Column'] > '']  

In [14]:
# save the transformed data
df_transformed_train.to_csv(lc.feature_transformation_output_table,index=False)
df_transformed_oot.to_csv(lc.transformed_features_oot_table_name,index=False)